# **Machice Translation  (English to Hindi)**

# Import Library 

In [1]:
pip install transformers datasets


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments


In [46]:
import pandas as pd

# load dataset 

In [10]:
# Load the IITB English-Hindi dataset
dataset = load_dataset("cfilt/iitb-english-hindi")



Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [43]:
dataset['train']

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [55]:
print(dataset['train']['translation'][0])

{'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}


In [48]:
train_df = pd.DataFrame(dataset['train'].select(range(5)))
print(test_df)

                                         translation
0  {'en': 'A black box in your car?', 'hi': 'आपकी...
1  {'en': 'As America's road planners struggle to...
2  {'en': 'The devices, which track every mile a ...
3  {'en': 'The usually dull arena of highway plan...
4  {'en': 'Libertarians have joined environmental...


# load Pre trained Model 

In [2]:
model_name = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# Data Preprocessing 

In [3]:
max_length = 128


In [50]:
def preprocess_function(examples):
    inputs = [ex["en"].lower() for ex in examples["translation"]]
    targets = [ex["hi"] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)

    # Tokenize and pad targets
    tokenized_targets = tokenizer(
        targets,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"  # Return PyTorch tensors
    )
    model_inputs["labels"] = tokenized_targets["input_ids"]

    return model_inputs


In [51]:
tokenized_datasets_validation = dataset['validation'].map(
    preprocess_function,
    batched= True,
    remove_columns=dataset["validation"].column_names,
    batch_size = 2
)
 
tokenized_datasets_test = dataset['test'].map(
    preprocess_function,
    batched= True,
    remove_columns=dataset["test"].column_names,
    batch_size = 2)

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [52]:
dataset_train_sampled = dataset['train'].shuffle(seed=42).select(range(10000))


In [53]:
tokenized_datasets_train = dataset_train_sampled.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    batch_size=2
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Model

In [22]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=100,
    predict_with_generate=True
)

In [23]:
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

In [24]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_validation,
    tokenizer=tokenizer,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.424700,1.472495
2,1.029500,1.367169
3,0.960800,1.315711
4,0.868200,1.271148
5,0.822800,1.239563
6,0.798200,1.214833
7,0.776300,1.197706
8,0.715800,1.179995
9,0.691500,1.167351
10,0.670700,1.160036


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=62500, training_loss=0.2906491841430664, metrics={'train_runtime': 13748.4972, 'train_samples_per_second': 72.735, 'train_steps_per_second': 4.546, 'total_flos': 2.032227316452557e+16, 'train_loss': 0.2906491841430664, 'epoch': 100.0})

In [26]:
results = trainer.evaluate(eval_dataset=tokenized_datasets_test)
print(results)

{'eval_loss': 2.0474207401275635, 'eval_runtime': 8.4217, 'eval_samples_per_second': 297.682, 'eval_steps_per_second': 18.642, 'epoch': 100.0}


In [27]:
model.save_pretrained('./fine-tuned-marianmt-en-hi')
tokenizer.save_pretrained('./fine-tuned-marianmt-en-hi')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


('./fine-tuned-marianmt-en-hi/tokenizer_config.json',
 './fine-tuned-marianmt-en-hi/special_tokens_map.json',
 './fine-tuned-marianmt-en-hi/vocab.json',
 './fine-tuned-marianmt-en-hi/source.spm',
 './fine-tuned-marianmt-en-hi/target.spm',
 './fine-tuned-marianmt-en-hi/added_tokens.json')

In [28]:
from transformers import MarianMTModel, MarianTokenizer

# Load the fine-tuned model and tokenizer
model_name = './fine-tuned-marianmt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model_test = MarianMTModel.from_pretrained(model_name)


/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [30]:
# Prepare test sentences
test_sentences = [
    "What is your name?",
    "I loved the movie.",
    "How are you?",
    "This is a test sentence for translation.",
]


In [31]:
# Tokenize the test sentences
inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True, max_length=128)


In [32]:
# Generate translations
translated_tokens = model_test.generate(**inputs)


In [33]:
# Decode the translations
translated_sentences = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]


In [34]:
# Print the results
for original, translated in zip(test_sentences, translated_sentences):
    print(f"Original: {original}")
    print(f"Translated: {translated}\n")


Original: What is your name?
Translated: तुम्हारा नाम क्या है?

Original: I loved the movie.
Translated: मैं प्रतिक्रिया को मता था।

Original: How are you?
Translated: तुम कैसे है?

Original: This is a test sentence for translation.
Translated: यह ्तान के लिए प्रकार की जाी वर्षा होती



In [42]:
test_sentence = ['who are you']
input = tokenizer(test_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
translated_token = model_test.generate(**input)
translated_sentence = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_token]
# Print the results
for original, translated in zip(test_sentence, translated_sentence):
    print(f"Original: {original}")
    print(f"Translated: {translated}\n")


Original: who are you
Translated: तुम वास् हैं



# download model 

In [35]:
import shutil
from IPython.display import FileLink

# Path to the folder you want to zip
folder_to_zip = '/kaggle/working/fine-tuned-marianmt-en-hi'
# Path for the output zip file
output_zip = '/kaggle/working/fine-tuned-marianmt-en-hi.zip'

# Zip the folder
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', folder_to_zip)

# Display the download link
FileLink(output_zip)


/kaggle/working/fine-tuned-marianmt-en-hi.zip